In [21]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 41 
num_classes = 3

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [22]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [3]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017-2018Rabi_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/2017-2018Rabi_VV1.csv', delimiter=',')

In [4]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
print(data.shape)
print(data1.shape)

(301, 27)
(301, 14)


In [5]:
print(data.max())
print(data1.max())

0.935826175
0.280879855


In [6]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(301, 27)
(301, 14)


In [7]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataRabi2017-2018.csv', names=colnames)

In [8]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat', 'Wheat

In [9]:
p = np.array(ctype)
print(p.shape)

(302,)


In [10]:
lbl1 = ctype
label = []
for i in range(1,302):
    if(lbl1[i] == "Wheat"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "Mustard"):
        label += [2]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [11]:
label = np.array(label)
print(label.shape)

(301,)


In [15]:
print(data.shape)
print(data1.shape)
data2 = np.concatenate((data,data1), axis=1)
print(data2.shape)

(301, 27)
(301, 14)
(301, 41)


In [16]:
x1=[]
x2=[]
x3=[]
l1=[]
l2=[]
l3=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)

print(l2.shape)

(142, 3)


In [17]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [18]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:240,:]
X_test = data2[241:,:]
y_train = label[0:240]
y_test = label[241:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(240, 41)
(60, 41)
(240,)
(60,)


In [19]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(240, 3)
(60, 3)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [23]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.79013453
Epoch: 2 cost = 0.54652699
Epoch: 3 cost = 0.42918777
Epoch: 4 cost = 0.35818545
Epoch: 5 cost = 0.32617826
Epoch: 6 cost = 0.31039108
Epoch: 7 cost = 0.29835838
Epoch: 8 cost = 0.28701612
Epoch: 9 cost = 0.27603362
Epoch: 10 cost = 0.26553037
Epoch: 11 cost = 0.25559058
Epoch: 12 cost = 0.24620360
Epoch: 13 cost = 0.23729550
Epoch: 14 cost = 0.22876558
Epoch: 15 cost = 0.22051563
Epoch: 16 cost = 0.21247765
Epoch: 17 cost = 0.20463026
Epoch: 18 cost = 0.19698794
Epoch: 19 cost = 0.18957214
Epoch: 20 cost = 0.18239243
Epoch: 21 cost = 0.17544748
Epoch: 22 cost = 0.16873668
Epoch: 23 cost = 0.16227309
Epoch: 24 cost = 0.15608933
Epoch: 25 cost = 0.15026214
Epoch: 26 cost = 0.14534005
Epoch: 27 cost = 0.14838458
Epoch: 28 cost = 0.18319060
Epoch: 29 cost = 0.18132687
Epoch: 30 cost = 0.14502562
Epoch: 31 cost = 0.21112632
Epoch: 32 cost = 0.36307921
Epoch: 33 cost = 0.26348453
Epoch: 34 cost = 0.15797303
Epoch: 35 cost = 0.13307780
Epoch: 36 cost = 0.11608812
E

Epoch: 313 cost = 0.01985048
Epoch: 314 cost = 0.01924171
Epoch: 315 cost = 0.01919253
Epoch: 316 cost = 0.01983167
Epoch: 317 cost = 0.01951923
Epoch: 318 cost = 0.01964033
Epoch: 319 cost = 0.01963248
Epoch: 320 cost = 0.01950630
Epoch: 321 cost = 0.01973345
Epoch: 322 cost = 0.02068453
Epoch: 323 cost = 0.01948138
Epoch: 324 cost = 0.01911201
Epoch: 325 cost = 0.01930954
Epoch: 326 cost = 0.02010775
Epoch: 327 cost = 0.01931713
Epoch: 328 cost = 0.01937615
Epoch: 329 cost = 0.01950499
Epoch: 330 cost = 0.01931247
Epoch: 331 cost = 0.06138777
Epoch: 332 cost = 1.04392713
Epoch: 333 cost = 0.06603034
Epoch: 334 cost = 0.06209956
Epoch: 335 cost = 0.02037897
Epoch: 336 cost = 0.01988154
Epoch: 337 cost = 0.01961784
Epoch: 338 cost = 0.01953154
Epoch: 339 cost = 0.01982421
Epoch: 340 cost = 0.01995091
Epoch: 341 cost = 0.01947139
Epoch: 342 cost = 0.01960843
Epoch: 343 cost = 0.01966111
Epoch: 344 cost = 0.01944811
Epoch: 345 cost = 0.01957286
Epoch: 346 cost = 0.01963704
Epoch: 347 cos

Epoch: 621 cost = 0.01713668
Epoch: 622 cost = 0.01748286
Epoch: 623 cost = 0.01750120
Epoch: 624 cost = 0.01758256
Epoch: 625 cost = 0.01759827
Epoch: 626 cost = 0.01715726
Epoch: 627 cost = 0.01760290
Epoch: 628 cost = 0.01738068
Epoch: 629 cost = 0.01708903
Epoch: 630 cost = 0.01710479
Epoch: 631 cost = 0.01742041
Epoch: 632 cost = 0.01734951
Epoch: 633 cost = 0.01748936
Epoch: 634 cost = 0.01772099
Epoch: 635 cost = 0.01767021
Epoch: 636 cost = 0.01724158
Epoch: 637 cost = 0.01693904
Epoch: 638 cost = 0.01746437
Epoch: 639 cost = 0.01777858
Epoch: 640 cost = 0.01746022
Epoch: 641 cost = 0.01705570
Epoch: 642 cost = 0.01701498
Epoch: 643 cost = 0.01717012
Epoch: 644 cost = 0.01749977
Epoch: 645 cost = 0.01832242
Epoch: 646 cost = 0.24751738
Epoch: 647 cost = 0.15157214
Epoch: 648 cost = 0.13267833
Epoch: 649 cost = 0.01836911
Epoch: 650 cost = 0.01812115
Epoch: 651 cost = 0.01790169
Epoch: 652 cost = 0.01787106
Epoch: 653 cost = 0.01774838
Epoch: 654 cost = 0.01771311
Epoch: 655 cos

Epoch: 948 cost = 0.01674325
Epoch: 949 cost = 0.01632801
Epoch: 950 cost = 0.01574574
Epoch: 951 cost = 0.01604588
Epoch: 952 cost = 0.01619800
Epoch: 953 cost = 0.01635263
Epoch: 954 cost = 0.01616259
Epoch: 955 cost = 0.01610984
Epoch: 956 cost = 0.01640875
Epoch: 957 cost = 0.01604001
Epoch: 958 cost = 0.01624863
Epoch: 959 cost = 0.01648824
Epoch: 960 cost = 0.01677905
Epoch: 961 cost = 0.01580228
Epoch: 962 cost = 0.01619894
Epoch: 963 cost = 0.01593924
Epoch: 964 cost = 0.01577085
Epoch: 965 cost = 0.01680120
Epoch: 966 cost = 0.01658443
Epoch: 967 cost = 0.01649217
Epoch: 968 cost = 0.01637418
Epoch: 969 cost = 0.01637673
Epoch: 970 cost = 0.01606632
Epoch: 971 cost = 0.01611477
Epoch: 972 cost = 0.01620936
Epoch: 973 cost = 0.01612266
Epoch: 974 cost = 0.01622520
Epoch: 975 cost = 0.01645911
Epoch: 976 cost = 0.01610909
Epoch: 977 cost = 0.01642019
Epoch: 978 cost = 0.01605028
Epoch: 979 cost = 0.01609490
Epoch: 980 cost = 0.01650462
Epoch: 981 cost = 0.01627962
Epoch: 982 cos

In [24]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [25]:
print(y_test.shape)

(60, 3)


In [26]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.78333336


In [21]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
print(res1)
print(res2)
print(res3)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 2
 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 2 1 2 2 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 2 1 1 1 2 2 1 1 1 1 1 1 0 1 1 1 1 1 2 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 0 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 2 2 2 1 1 1 1 2 2 1 1 0 1 1 2 2 1 2 1 2 2 1 2 2 1 1 2 2 2 1 1 1 1 2 2 1
 1 2]


In [22]:
# confusion matrix
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
confusion.append([x1,x2,x3])
x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
confusion.append([x1,x2,x3])

confusion=np.array(confusion)
print(confusion)

[[112   7   1]
 [  5 125  12]
 [  1  20  18]]


In [21]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:210,:]
X_test = data2[210:,:]
y_train = label[0:210]
y_test = label[210:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(210, 41)
(91, 41)
(210,)
(91,)


In [22]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(210, 3)
(91, 3)


In [25]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.42742094
Epoch: 2 cost = 0.42135174
Epoch: 3 cost = 0.41620045
Epoch: 4 cost = 0.41156487
Epoch: 5 cost = 0.40710450
Epoch: 6 cost = 0.40255917
Epoch: 7 cost = 0.39774728
Epoch: 8 cost = 0.39255254
Epoch: 9 cost = 0.38690797
Epoch: 10 cost = 0.38078314
Epoch: 11 cost = 0.37417670
Epoch: 12 cost = 0.36711429
Epoch: 13 cost = 0.35964875
Epoch: 14 cost = 0.35186043
Epoch: 15 cost = 0.34385304
Epoch: 16 cost = 0.33574531
Epoch: 17 cost = 0.32765790
Epoch: 18 cost = 0.31969858
Epoch: 19 cost = 0.31194830
Epoch: 20 cost = 0.30444924
Epoch: 21 cost = 0.29719779
Epoch: 22 cost = 0.29014981
Epoch: 23 cost = 0.28324634
Epoch: 24 cost = 0.27645588
Epoch: 25 cost = 0.26980331
Epoch: 26 cost = 0.26335440
Epoch: 27 cost = 0.25716721
Epoch: 28 cost = 0.25125666
Epoch: 29 cost = 0.24559832
Epoch: 30 cost = 0.24015334
Epoch: 31 cost = 0.23488696
Epoch: 32 cost = 0.22977115
Epoch: 33 cost = 0.22477758
Epoch: 34 cost = 0.21987352
Epoch: 35 cost = 0.21502581
Epoch: 36 cost = 0.21021050
E

Epoch: 322 cost = 0.00735475
Epoch: 323 cost = 0.00742703
Epoch: 324 cost = 0.00723340
Epoch: 325 cost = 0.00738031
Epoch: 326 cost = 0.00746181
Epoch: 327 cost = 0.00723530
Epoch: 328 cost = 0.00734881
Epoch: 329 cost = 0.00769153
Epoch: 330 cost = 0.00747692
Epoch: 331 cost = 0.00721630
Epoch: 332 cost = 0.00720503
Epoch: 333 cost = 0.00724133
Epoch: 334 cost = 0.00728720
Epoch: 335 cost = 0.00725454
Epoch: 336 cost = 0.00708660
Epoch: 337 cost = 0.00719490
Epoch: 338 cost = 0.00744651
Epoch: 339 cost = 0.00721431
Epoch: 340 cost = 0.00717257
Epoch: 341 cost = 0.00743213
Epoch: 342 cost = 0.00723405
Epoch: 343 cost = 0.00745163
Epoch: 344 cost = 0.00742707
Epoch: 345 cost = 0.00702487
Epoch: 346 cost = 0.00710625
Epoch: 347 cost = 0.00733251
Epoch: 348 cost = 0.00733212
Epoch: 349 cost = 0.00704840
Epoch: 350 cost = 0.00736599
Epoch: 351 cost = 0.00764681
Epoch: 352 cost = 0.00718429
Epoch: 353 cost = 0.00714256
Epoch: 354 cost = 0.00728700
Epoch: 355 cost = 0.00736232
Epoch: 356 cos

Epoch: 627 cost = 0.00432484
Epoch: 628 cost = 0.00438675
Epoch: 629 cost = 0.00426105
Epoch: 630 cost = 0.00417661
Epoch: 631 cost = 0.00422647
Epoch: 632 cost = 0.00433048
Epoch: 633 cost = 0.00432509
Epoch: 634 cost = 0.00421738
Epoch: 635 cost = 0.00426697
Epoch: 636 cost = 0.00430167
Epoch: 637 cost = 0.00434037
Epoch: 638 cost = 0.00426727
Epoch: 639 cost = 0.00423957
Epoch: 640 cost = 0.00434594
Epoch: 641 cost = 0.00442130
Epoch: 642 cost = 0.00420036
Epoch: 643 cost = 0.00416198
Epoch: 644 cost = 0.00426204
Epoch: 645 cost = 0.00439105
Epoch: 646 cost = 0.00422293
Epoch: 647 cost = 0.00418280
Epoch: 648 cost = 0.00433773
Epoch: 649 cost = 0.00438713
Epoch: 650 cost = 0.00427359
Epoch: 651 cost = 0.00424544
Epoch: 652 cost = 0.00419859
Epoch: 653 cost = 0.00413543
Epoch: 654 cost = 0.00420723
Epoch: 655 cost = 0.00425902
Epoch: 656 cost = 0.00439217
Epoch: 657 cost = 0.00440980
Epoch: 658 cost = 0.00424268
Epoch: 659 cost = 0.00438438
Epoch: 660 cost = 0.00427902
Epoch: 661 cos

Epoch: 944 cost = 0.00402529
Epoch: 945 cost = 0.00410631
Epoch: 946 cost = 0.00428523
Epoch: 947 cost = 0.00408796
Epoch: 948 cost = 0.00410332
Epoch: 949 cost = 0.00369531
Epoch: 950 cost = 0.00249710
Epoch: 951 cost = 0.00156164
Epoch: 952 cost = 0.00094534
Epoch: 953 cost = 0.00066663
Epoch: 954 cost = 0.00057778
Epoch: 955 cost = 0.00053391
Epoch: 956 cost = 0.00057534
Epoch: 957 cost = 0.00048118
Epoch: 958 cost = 0.00049215
Epoch: 959 cost = 0.00055928
Epoch: 960 cost = 0.00049331
Epoch: 961 cost = 0.00052090
Epoch: 962 cost = 0.00060557
Epoch: 963 cost = 0.00057727
Epoch: 964 cost = 0.00055333
Epoch: 965 cost = 0.00060625
Epoch: 966 cost = 0.00063862
Epoch: 967 cost = 0.00047197
Epoch: 968 cost = 0.00066520
Epoch: 969 cost = 0.00052872
Epoch: 970 cost = 0.00035774
Epoch: 971 cost = 0.00045161
Epoch: 972 cost = 0.00047756
Epoch: 973 cost = 0.00050561
Epoch: 974 cost = 0.00051757
Epoch: 975 cost = 0.00060020
Epoch: 976 cost = 0.00061177
Epoch: 977 cost = 0.00061483
Epoch: 978 cos

In [27]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.84615386
